In [1]:
# Importing Libraries


import pandas as pd
import numpy as np
import pyodbc
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from itertools import combinations, groupby
from collections import Counter




In [2]:
# connecting to SQL server

pyodbc.connect

cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=DESKTOP-TI5OC9C;"
            "Database=AdventureWorks2012;"
            "Trusted_Connection=yes;")


cnxn = pyodbc.connect(cnxn_str)


In [3]:
# Importing data

dataset = pd.read_sql("select [A].[Name] as [Item],[D].[OrderQty] as [Quantity],[E].[SalesOrderID] as [TransactionID],[F].[Name] as [Region] from [Production].[Product] as [A]LEFT JOIN [Production].[ProductSubcategory] as [B] on [A].[ProductSubcategoryID] = [B].[ProductSubcategoryID]LEFT JOIN [Production].[ProductCategory] as [C] on [B].[ProductCategoryID] = [C].[ProductCategoryID]LEFT JOIN [Sales].[SalesOrderDetail] as [D] on [A].[ProductID] = [D].[ProductID]LEFT JOIN [Sales].[SalesOrderHeader] as [E] on [D].[SalesOrderID] = [E].[SalesOrderID]LEFT JOIN [Sales].[SalesTerritory] as [F] on [E].[TerritoryID] = [F].[TerritoryID] where OnlineOrderFlag = 0 " , cnxn)
dataset.head(10)

,Item,Quantity,TransactionID,Region
0,"Mountain-100 Black, 42",1,43659,Southeast
1,"Mountain-100 Black, 44",3,43659,Southeast
2,"Mountain-100 Black, 48",1,43659,Southeast
3,"Mountain-100 Silver, 38",1,43659,Southeast
4,"Mountain-100 Silver, 42",1,43659,Southeast
5,"Mountain-100 Silver, 44",2,43659,Southeast
6,"Mountain-100 Silver, 48",1,43659,Southeast
7,"Long-Sleeve Logo Jersey, M",3,43659,Southeast
8,"Long-Sleeve Logo Jersey, XL",1,43659,Southeast
9,"Mountain Bike Socks, M",6,43659,Southeast


### Data Preparations

In [11]:
dataset.shape

(60919, 4)

In [12]:
dataset['Item'] = dataset['Item'].str.strip() # removing spaces
dataset.head()

,Item,Quantity,TransactionID,Region
0,"Mountain-100 Black, 42",1,43659,Southeast
1,"Mountain-100 Black, 44",3,43659,Southeast
2,"Mountain-100 Black, 48",1,43659,Southeast
3,"Mountain-100 Silver, 38",1,43659,Southeast
4,"Mountain-100 Silver, 42",1,43659,Southeast


In [13]:
dataset.dropna(axis=0, subset=['TransactionID'], inplace=True)# removing duplicates
dataset.head()

,Item,Quantity,TransactionID,Region
0,"Mountain-100 Black, 42",1,43659,Southeast
1,"Mountain-100 Black, 44",3,43659,Southeast
2,"Mountain-100 Black, 48",1,43659,Southeast
3,"Mountain-100 Silver, 38",1,43659,Southeast
4,"Mountain-100 Silver, 42",1,43659,Southeast


In [14]:
dataset['TransactionID'] = dataset['TransactionID'].astype('str')# converting to string
dataset.head()

,Item,Quantity,TransactionID,Region
0,"Mountain-100 Black, 42",1,43659,Southeast
1,"Mountain-100 Black, 44",3,43659,Southeast
2,"Mountain-100 Black, 48",1,43659,Southeast
3,"Mountain-100 Silver, 38",1,43659,Southeast
4,"Mountain-100 Silver, 42",1,43659,Southeast


In [15]:
dataset.shape

(60919, 4)

In [16]:
dataset['Region'].value_counts()

Southwest         13379
Canada            11444
Northwest          7872
Southeast          5937
Central            5812
Northeast          5809
France             3530
United Kingdom     3520
Germany            1903
Australia          1713
Name: Region, dtype: int64

In [17]:
print(dataset.isnull().values.any())

False


In [18]:
missing_data = dataset.isnull().sum()
missing_data

Item             0
Quantity         0
TransactionID    0
Region           0
dtype: int64

In [19]:
unique_values= dataset.nunique()
print ("The unique values for each Column are: ")

print (unique_values)

The unique values for each Column are: 
Item              250
Quantity           41
TransactionID    3806
Region             10
dtype: int64


In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60919 entries, 0 to 60918
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Item           60919 non-null  object
 1   Quantity       60919 non-null  int64 
 2   TransactionID  60919 non-null  object
 3   Region         60919 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [21]:
data_plus = dataset[dataset['Quantity']>=0]
data_plus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60919 entries, 0 to 60918
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Item           60919 non-null  object
 1   Quantity       60919 non-null  int64 
 2   TransactionID  60919 non-null  object
 3   Region         60919 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [22]:
## selecting All Regions group ['Transaction ID', 'Product_Name'] and sum by Quality

market_basket = (data_plus
                .groupby(['TransactionID', 'Item'])['Quantity']
                .sum().unstack().reset_index().fillna(0)
                .set_index('TransactionID'))


In [23]:
# encoding as per associatios Rule-all positive values to 1 and rest to 0

def encode_data(datapoint):
    if datapoint <= 0:
        return 0
    if datapoint >= 1:
        return 1

In [24]:
market_basket = market_basket.applymap(encode_data)
market_basket

Item,AWC Logo Cap,Bike Wash - Dissolver,Cable Lock,Chain,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Front Brakes,Front Derailleur,"Full-Finger Gloves, L",...,"Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S","Women's Tights, L","Women's Tights, M","Women's Tights, S"
TransactionID,,,,,,,,,,,,,,,,,,,,,
43659,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43660,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43661,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43662,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71948,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71949,1,1,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
71950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


### Trainin Model

In [27]:
# Generating Frequent Itemsets

MyFrequentItemsets = apriori(market_basket, min_support=0.08, use_colnames=True)

In [28]:
# Creating Association Rules

rules = association_rules(MyFrequentItemsets, metric="lift", min_threshold= 2.2)
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(AWC Logo Cap),(Bike Wash - Dissolver),0.313190,0.110089,0.098266,0.313758,2.850034,0.063787,1.296789
1,(Bike Wash - Dissolver),(AWC Logo Cap),0.110089,0.313190,0.098266,0.892601,2.850034,0.063787,6.394967
2,(AWC Logo Cap),"(Classic Vest, M)",0.313190,0.093537,0.086180,0.275168,2.941822,0.056885,1.250584
3,"(Classic Vest, M)",(AWC Logo Cap),0.093537,0.313190,0.086180,0.921348,2.941822,0.056885,8.732302
4,(AWC Logo Cap),"(Classic Vest, S)",0.313190,0.135050,0.109038,0.348154,2.577968,0.066742,1.326925
5,"(Classic Vest, S)",(AWC Logo Cap),0.135050,0.313190,0.109038,0.807393,2.577968,0.066742,3.565864
6,(AWC Logo Cap),"(Half-Finger Gloves, M)",0.313190,0.154230,0.142932,0.456376,2.959057,0.094629,1.555799
7,"(Half-Finger Gloves, M)",(AWC Logo Cap),0.154230,0.313190,0.142932,0.926746,2.959057,0.094629,9.375759
8,(AWC Logo Cap),"(Half-Finger Gloves, S)",0.313190,0.103783,0.092223,0.294463,2.837282,0.059719,1.270262
9,"(Half-Finger Gloves, S)",(AWC Logo Cap),0.103783,0.313190,0.092223,0.888608,2.837282,0.059719,6.165683


### Making Recommendaions

In [30]:
market_basket['AWC Logo Cap'].sum()

1192

In [31]:
market_basket['Classic Vest, M'].sum()

356

#### For the item 'Touring Tire Tube'  sold 1488 and out of which 935 'Touring Tire' has been bought which shows a good combination and is best to recommend it to the one who is buying 'Touring Tire Tube'

In [36]:
market_basket['Long-Sleeve Logo Jersey, L'].sum()

1183

In [35]:
market_basket['AWC Logo Cap'].sum()

1192